In [ ]:
import pandas as pd
import mysql.connector as connection

In [ ]:
raw_sales_data = connection.connect(host="devcodecamp-mysql.c5p7yvjq1h8y.us-east-2.rds.amazonaws.com", database="salesdb", user="root", password="D3v$tudent", use_pure=True)